# Taking Cael's location values

### *Import Packages and Modules*
***

In [2]:
%reset
import xarray as xr
import numpy as np
import netCDF4 as nc
import glob
import re
import matplotlib.pyplot as plt
import matplotlib
import warnings
from matplotlib import *
from netCDF4 import Dataset
from scipy import stats
from matplotlib.colors import LinearSegmentedColormap
from pylr2 import regress2
import itertools
import statsmodels.api as sm
import time
from operator import itemgetter
import csv
warnings.filterwarnings("ignore")

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [3]:
sys.path.append('.')
from major_axis_regression import regress2
from parula import parula

In [4]:
parula_map = parula()

***
### *Import Data*


In [8]:
BBCael = np.genfromtxt(open("../data/CAELsedimentObsMappedToModelGrid.csv"), delimiter=",")

In [9]:
BBCael.shape

(7160, 12)

In [10]:
bb_month = ((BBCael[:,2].astype(int))-1).astype(list)
bb_lat = BBCael[:,9]
bb_lon = BBCael[:,10]
bb_z = BBCael[:,11]

In [11]:
np.min(bb_month[1:7160])

0

In [12]:
ds_diad = xr.open_mfdataset('/gpfs/home/vsz14eru/scratch/models/ModelRuns/TOM12_JG_OCCA/ORCA2_1m_20000101_20001231_diad_T.nc')

In [13]:
EXP_xr = (ds_diad['EXP']*12*1e3*86400)
ExpCO3_xr = (ds_diad['ExpCO3']*12*1e3*86400)
sinksil_xr = (ds_diad['sinksil']*28*1e3*86400)

EXP = EXP_xr.values
ExpCO3 = ExpCO3_xr.values
sinksil = sinksil_xr.values

In [14]:
EXP.shape

(12, 31, 149, 182)

In [15]:
print(np.max(bb_lat[1:]))
print(np.max(bb_lon[1:]))


146.0
177.0


In [16]:
print(bb_month[1:10])
print(bb_z[1:10])
print(bb_lon[1:10]) 
print(bb_lat[1:10])

[6 7 8 8 9 9 10 11 11]
[26. 26. 26. 26. 26. 26. 26. 26. 26.]
[51. 51. 51. 51. 51. 51. 51. 51. 51.]
[116. 116. 116. 116. 116. 116. 116. 116. 116.]


In [17]:
depths = ((4.999938, 15.00029, 25.00176, 35.00541, 45.01332, 55.0295, 
               65.06181, 75.12551, 85.25037, 95.49429, 105.9699, 116.8962, 128.6979, 
               142.1953, 158.9606, 181.9628, 216.6479, 272.4767, 364.303, 511.5348, 
               732.2009, 1033.217, 1405.698, 1830.885, 2289.768, 2768.242, 3257.479, 
               3752.442, 4250.401, 4749.913, 5250.227))


In [18]:
from operator import itemgetter
new_depths_SO = np. asarray((itemgetter(*bb_z[1:].astype(int))(depths)))


In [19]:
new_depths_SO.astype(int)

array([3257, 3257, 3257, ..., 1405,  158, 1405])

In [20]:
coords = (6,26,116,51)
x = EXP[coords]
print(x)

9.227638668380678


In [21]:
np.nanmax(bb_lon)

177.0

In [22]:
coords = np.stack((bb_month[1:7160], bb_z[1:7160], bb_lat[1:7160], bb_lon[1:7160]), axis=0).astype(int)

In [23]:
coords[:,1]

array([  7,  26, 116,  51])

In [24]:
print(EXP[coords[0,50], coords[1,50], coords[2,50], coords[3,50]])

5.795737775042653


In [25]:
EXP = np.ones((7159))*np.nan
for i in np.arange(0,7159,1):
    EXP[i] = EXP_xr[coords[0,i], coords[1,i], coords[2,i], coords[3,i]]
    

In [26]:
print(EXP)

[ 9.22763867  6.6135674   6.35536434 ...  0.76297764 58.53304099
 31.41787928]


In [27]:
EXPCO3 = np.ones((7159))*np.nan
for i in np.arange(0,7159,1):
    EXPCO3[i] = ExpCO3_xr[coords[0,i], coords[1,i], coords[2,i], coords[3,i]]

In [28]:
sinksil = np.ones((7159))*np.nan
for i in np.arange(0,7159,1):
    sinksil[i] = sinksil_xr[coords[0,i], coords[1,i], coords[2,i], coords[3,i]]

In [29]:
values = np.stack((coords[0,:], coords[1,:], coords[2,:], coords[3,:], EXP, EXPCO3, sinksil, new_depths_SO),axis = 0)

In [30]:
values[:,6000]

array([   8.        ,   27.        ,   76.        ,  135.        ,
          6.2935851 ,    4.45429307,    4.00798753, 3752.442     ])

In [31]:
    depths = ((4.999938, 15.00029, 25.00176, 35.00541, 45.01332, 55.0295, 
               65.06181, 75.12551, 85.25037, 95.49429, 105.9699, 116.8962, 128.6979, 
               142.1953, 158.9606, 181.9628, 216.6479, 272.4767, 364.303, 511.5348, 
               732.2009, 1033.217, 1405.698, 1830.885, 2289.768, 2768.242, 3257.479, 
               3752.442, 4250.401, 4749.913, 5250.227))


In [32]:
    from operator import itemgetter
    new_depths_SO = np. asarray((itemgetter(*bb_z[1:].astype(int))(depths)))


In [33]:
new_depths_SO.astype(int)

array([3257, 3257, 3257, ..., 1405,  158, 1405])

In [34]:
values.shape

(8, 7159)

In [35]:
np.save('../data/OCCA_masked2obs/BB_LOC_OCCA_JG.npy', values)

### Southern Ocean 

In [36]:
SO_OCCA1 = np.ones((8,7159))*np.nan

In [37]:
for i in np.arange(0,8,1):
    for j in np.arange(0,7159,1):
        if values[2,j] < 50:
            SO_OCCA1[i,j] = values[i,j]

In [38]:
values[2,4000:4200]

array([108., 108., 108., 108., 108., 112., 112., 112., 112., 112., 112.,
       112., 112., 112., 112., 112., 112., 112., 112., 112., 112., 112.,
       112., 112., 112., 112., 114., 114., 114., 114., 114., 114., 114.,
       114., 114., 114., 114., 114., 114., 114., 114., 114., 114., 114.,
       114., 114., 114., 114., 114., 114., 114., 114., 114., 114., 114.,
       114., 114., 114., 114., 114., 114., 114., 114., 114., 114., 114.,
       114., 114.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,
        34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,
        34.,  34.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,
        30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,
        30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,
        30.,  30.,  30.,  30.,  30.,  30.,  27.,  27.,  27.,  27.,  27.,
        27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,
        27.,  27.,  27.,  27.,  27.,  24.,  24.,  2

In [39]:
SO_OCCA1[:,4203]

array([1.00000000e+01, 1.60000000e+01, 4.00000000e+00, 5.20000000e+01,
       9.16043817e-01, 6.29338105e-04, 2.36521113e-01, 2.16647900e+02])

In [40]:
SO_OCCA = SO_OCCA1[:, ~np.isnan(SO_OCCA1).any(axis=0)]

In [41]:
print(SO_OCCA.shape)

(8, 768)


In [42]:
np.save('../data/OCCA_masked2obs/SO_BB_OCCA.npy', SO_OCCA)

### North Pacific

In [43]:
NP_OCCA1 = np.ones((8,7159))*np.nan

In [44]:
for i in np.arange(0,8,1):
    for j in np.arange(0,7159,1):
        if values[2,j] > 100 and (0 < values[3,j] < 80):
            NP_OCCA1[i,j] = values[i,j]

In [45]:
values[2,4000:4200]

array([108., 108., 108., 108., 108., 112., 112., 112., 112., 112., 112.,
       112., 112., 112., 112., 112., 112., 112., 112., 112., 112., 112.,
       112., 112., 112., 112., 114., 114., 114., 114., 114., 114., 114.,
       114., 114., 114., 114., 114., 114., 114., 114., 114., 114., 114.,
       114., 114., 114., 114., 114., 114., 114., 114., 114., 114., 114.,
       114., 114., 114., 114., 114., 114., 114., 114., 114., 114., 114.,
       114., 114.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,
        34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,
        34.,  34.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,
        30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,
        30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,
        30.,  30.,  30.,  30.,  30.,  30.,  27.,  27.,  27.,  27.,  27.,
        27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,
        27.,  27.,  27.,  27.,  27.,  24.,  24.,  2

In [46]:
NP_OCCA1[:,4200]

array([nan, nan, nan, nan, nan, nan, nan, nan])

In [47]:
NP_OCCA = NP_OCCA1[:, ~np.isnan(NP_OCCA1).any(axis=0)]

In [48]:
print(NP_OCCA.shape)

(8, 1919)


In [49]:
np.save('../data/OCCA_masked2obs/NP_BB_OCCA.npy', NP_OCCA)

### North Atlantic

In [50]:
NA_OCCA1 = np.ones((8,7159))*np.nan

In [51]:
for i in np.arange(0,8,1):
    for j in np.arange(0,7159,1):
        if values[2,j] > 99 and (100 < values[3,j] < 160):
            NA_OCCA1[i,j] = values[i,j]

In [52]:
values[2,4000:4200]

array([108., 108., 108., 108., 108., 112., 112., 112., 112., 112., 112.,
       112., 112., 112., 112., 112., 112., 112., 112., 112., 112., 112.,
       112., 112., 112., 112., 114., 114., 114., 114., 114., 114., 114.,
       114., 114., 114., 114., 114., 114., 114., 114., 114., 114., 114.,
       114., 114., 114., 114., 114., 114., 114., 114., 114., 114., 114.,
       114., 114., 114., 114., 114., 114., 114., 114., 114., 114., 114.,
       114., 114.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,
        34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,
        34.,  34.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,
        30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,
        30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,
        30.,  30.,  30.,  30.,  30.,  30.,  27.,  27.,  27.,  27.,  27.,
        27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,
        27.,  27.,  27.,  27.,  27.,  24.,  24.,  2

In [53]:
NA_OCCA1[:,4200]

array([nan, nan, nan, nan, nan, nan, nan, nan])

In [54]:
NA_OCCA = NA_OCCA1[:, ~np.isnan(NA_OCCA1).any(axis=0)]

In [55]:
print(NA_OCCA.shape)

(8, 1106)


In [56]:
np.save('../data/OCCA_masked2obs/NA_BB_OCCA.npy', NA_OCCA)

### Tropical Atlantic

In [57]:
TA_OCCA1 = np.ones((8,7159))*np.nan

In [58]:
for i in np.arange(0,8,1):
    for j in np.arange(0,7159,1):
        if (49 < values[2,j] < 101) and (100 < values[3,j] < 180):
            TA_OCCA1[i,j] = values[i,j]

In [59]:
values[2,4000:4200]

array([108., 108., 108., 108., 108., 112., 112., 112., 112., 112., 112.,
       112., 112., 112., 112., 112., 112., 112., 112., 112., 112., 112.,
       112., 112., 112., 112., 114., 114., 114., 114., 114., 114., 114.,
       114., 114., 114., 114., 114., 114., 114., 114., 114., 114., 114.,
       114., 114., 114., 114., 114., 114., 114., 114., 114., 114., 114.,
       114., 114., 114., 114., 114., 114., 114., 114., 114., 114., 114.,
       114., 114.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,
        34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,
        34.,  34.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,
        30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,
        30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,
        30.,  30.,  30.,  30.,  30.,  30.,  27.,  27.,  27.,  27.,  27.,
        27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,
        27.,  27.,  27.,  27.,  27.,  24.,  24.,  2

In [60]:
TA_OCCA1[:,4200]

array([nan, nan, nan, nan, nan, nan, nan, nan])

In [61]:
TA_OCCA = TA_OCCA1[:, ~np.isnan(TA_OCCA1).any(axis=0)]

In [62]:
print(TA_OCCA.shape)

(8, 1659)


In [63]:
np.save('../data/OCCA_masked2obs/TA_BB_OCCA.npy', TA_OCCA)

### Tropical Pacific

In [64]:
TP_OCCA1 = np.ones((8,7159))*np.nan

In [65]:
for i in np.arange(0,8,1):
    for j in np.arange(0,7159,1):
        if 49 < values[2,j] < 101 and (23 < values[3,j] < 86):
            TP_OCCA1[i,j] = values[i,j]

In [66]:
values[2,4000:4200]

array([108., 108., 108., 108., 108., 112., 112., 112., 112., 112., 112.,
       112., 112., 112., 112., 112., 112., 112., 112., 112., 112., 112.,
       112., 112., 112., 112., 114., 114., 114., 114., 114., 114., 114.,
       114., 114., 114., 114., 114., 114., 114., 114., 114., 114., 114.,
       114., 114., 114., 114., 114., 114., 114., 114., 114., 114., 114.,
       114., 114., 114., 114., 114., 114., 114., 114., 114., 114., 114.,
       114., 114.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,
        34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,  34.,
        34.,  34.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,
        30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,
        30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,
        30.,  30.,  30.,  30.,  30.,  30.,  27.,  27.,  27.,  27.,  27.,
        27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,
        27.,  27.,  27.,  27.,  27.,  24.,  24.,  2

In [67]:
TP_OCCA1[:,4200]

array([nan, nan, nan, nan, nan, nan, nan, nan])

In [68]:
TP_OCCA = TP_OCCA1[:, ~np.isnan(TP_OCCA1).any(axis=0)]

In [69]:
print(TP_OCCA.shape)

(8, 1707)


In [70]:
np.save('../data/OCCA_masked2obs/TP_BB_OCCA.npy', TP_OCCA)